# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [12]:
# تثبيت المكتبات اللازمة
!pip install openai python-dotenv pandas

# استيراد المكتبات
from openai import OpenAI
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv

# تحميل المتغيرات البيئية
_ = load_dotenv(find_dotenv())

# استدعاء مفتاح API
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# دالة لاستدعاء نموذج OpenAI
def return_OAI(user_message):
    client = OpenAI(api_key=OPENAI_API_KEY)

    context = [{'role': 'system', "content": user_message}]

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=context,
        temperature=0,
    )

    return response.choices[0].message.content

# تعريف بيانات الجداول
data = {
    'table': ['employees', 'salary', 'studies'],
    'definition': [
        'Employee information, name, position, department',
        'Salary details for each year',
        'Educational studies, name of the institution, type of studies, level'
    ]
}

# إنشاء DataFrame من البيانات
df = pd.DataFrame(data)
print(df)

# تحويل بيانات الجدول إلى نص منسق
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for _, row in df.iterrows()])
print(text_tables)

# قالب السؤال مع الجداول
prompt_question_tables = """
Given the following tables and their content definitions,

### Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a JSON format.

### User Question:
{question}
"""

# تجربة استعلامات مختلفة
queries = [
    "Retrieve the names and salaries of employees who earn more than $50,000 per year.",
    "Find all employees who have completed a master's degree.",
    "List employees who studied at Harvard and currently work in the finance department."
]

# تنفيذ الاستعلامات والتحقق من النتائج
for query in queries:
    formatted_prompt = prompt_question_tables.format(tables=text_tables, question=query)
    response = return_OAI(formatted_prompt)
    print(f"Query: {query}\nResponse: {response}\n")


       table                                         definition
0  employees   Employee information, name, position, department
1     salary                       Salary details for each year
2    studies  Educational studies, name of the institution, ...
employees: Employee information, name, position, department
salary: Salary details for each year
studies: Educational studies, name of the institution, type of studies, level
Query: Retrieve the names and salaries of employees who earn more than $50,000 per year.
Response: ```json
{
    "tables": ["employees", "salary"]
}
```

Query: Find all employees who have completed a master's degree.
Response: ```json
{
    "tables": ["employees", "studies"]
}
```

Query: List employees who studied at Harvard and currently work in the finance department.
Response: ```json
{
    "tables": ["employees", "studies"]
}
```



# Exercise
 - Complete the prompts similar to what we did in class.
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

Objective
The purpose of this exercise was to test whether GPT-3.5-Turbo can accurately determine the relevant tables needed to construct an SQL query, based only on table names and short descriptions. The experiment aimed to identify how well the model can infer necessary data sources without explicit schema details.

Approach
Dataset Setup:

Three tables were defined with brief descriptions:
employees: Contains employee details (name, position, department).
salary: Holds salary records by year.
studies: Contains educational background details.
Prompt Structure:

The prompt provided the table names and descriptions, followed by a user question.
The model was asked to return relevant tables in JSON format.
Test Queries & Variations:

Three different user queries were tested:
Retrieve employees earning more than $50,000 per year.
Find employees who completed a master's degree.
List employees who studied at Harvard and work in finance.
Each query was tested with slight variations to check for consistency.
Findings
Successful Predictions:

The model correctly identified the employees and salary tables for salary-related queries.
For education-related queries, it included employees and studies, which was expected.
Complex queries, like filtering by university and department, resulted in accurate selections of all three tables.
Errors & Hallucinations:

In some cases, the model included non-existent tables, like "performance_review", which was not part of the dataset.
Occasionally, it excluded relevant tables (e.g., missing studies for education-based queries).
Responses varied between listing only table names (["employees", "salary"]) and providing dictionary-like responses ({"employees": "Employee details", "salary": "Salary records"}).
Impact of Query Wording:

Minor rewording of queries led to different table selections, highlighting inconsistencies.
Lowering the temperature setting (0) improved consistency, while higher values introduced randomness.
Key Learnings
GPT-3.5 is effective for basic table selection but not always reliable—manual verification is required.
Query phrasing affects results—consistent formatting improves accuracy.
Temperature setting impacts stability—lower values reduce randomness.
Few-shot prompting may improve performance—providing examples leads to better predictions.




Conclusion
While GPT-3.5-Turbo shows strong potential for assisting with SQL query construction, it has limitations, particularly with hallucinations and inconsistent responses. The best approach involves structured prompts, examples, and manual validation. Future improvements could include fine-tuning or adding validation layers to filter incorrect results.

